In [ ]:
!pip install tweepy
!pip install textblob
!python -m textblob.download_corpora
!pip install nltk
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install wordcloud

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# -*- coding: utf-8 -*-
import re
import csv
import tweepy
import unicodedata
import json
import nltk
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.misc import imread
import matplotlib.pyplot as plt

In [ ]:
#Class Definiton for the keywords that will be used to fetch the tweets
class keywords:
    def __init__(self):
        self.keywords = ['AlaskaAir','Allegiant','AmericanAir','Delta','FlyFrontier','HawaiianAir','@united','JetBlue','SouthwestAir','SpiritAirlines','VirginAmerica','SunCountryAir']
        
    def getKeyWords(self):
        return self.keywords

In [ ]:
#Class Definition to pull the twitter tweets
class pullData:      
    def __init__(self,key,secret,maxTweets,tweetsPerQry,keywords):
        self.key = key   
        self.secret = secret
        self.maxTweets = maxTweets
        self.tweetsPerQry = tweetsPerQry          
        self.keywords = keywords
        self.api = ''
        self.auth = ''
        
    def printParams(self):
        print('Parameters set to...')
        print('key...',self.key)
        print('secret...',self.secret)
        print('maxTweets...',self.maxTweets)
        print('tweetsPerQry...',self.tweetsPerQry)
        print('keywords...',self.keywords)
        
    def connect(self):
        self.auth = tweepy.AppAuthHandler(self.key,self.secret )  
        self.api = tweepy.API(self.auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
        return True
        
        if (not self.api):
            print ("Can't Authenticate")
            return False
        
    def downloadData(self):
        for word in self.keywords:
            print( "Downloading Tweets for the keyword: ", word )
            fName = 'tweets_' + word + '.txt'
            sinceId = None
            max_id = -1
            tweetCount = 0
            tweet_dict = []
    
            print("Downloading max {0} tweets".format(maxTweets))

            while tweetCount < self.maxTweets:
                try:
                    if (max_id <= 0):
                        if (not sinceId):   
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry)
                        else:
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,since_id=sinceId)
                    else:
                        if (not sinceId):
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,
                                    max_id=str(max_id - 1))
                        else:
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,
                                    max_id=str(max_id - 1),since_id=sinceId)
                    if not new_tweets:
                        print("No more tweets found")
                        break            
                        
                    for tweet in new_tweets:
                        tweet_dict.append(tweet._json)
 
                    tweetCount += len(new_tweets)
                    print("Downloaded {0} tweets".format(tweetCount))
                    max_id = new_tweets[-1].id
                
                except tweepy.TweepError as e:
                    # Just exit if any error
                    print("some error : " + str(e))
                    break
            
            with open(fName, 'w', encoding='utf8', errors='replace') as f:   
                json.dump(tweet_dict, f, ensure_ascii=False)
    

In [ ]:
import sys

consumer_key = 'bppIV0LkQUIARDug4b8Sij8pm'
consumer_secret = '5ofpZtxL1CsDmfbF93Qh0EcPOHUVP3ZSF6CGhbkk2ki2fFWNFL'
maxTweets = 100000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

keys = keywords()
words = keys.getKeyWords()  # this is what we're searching for

pulldata = pullData(consumer_key,consumer_secret,maxTweets,tweetsPerQry,words)

if ( pulldata.connect() == False ):
    print ( "Connecting to the twitter API Failed")
    sys.exit(1)
pulldata.printParams()
    
pulldata.downloadData()



In [ ]:
#Class definition for JsonParser

class JsonParser:
    def loadData(self,fname):
        with open(fname, encoding='utf8', errors='replace') as json_data:
            d = json.load(json_data)
        return d
        

In [ ]:
#Class definition to Clean the tweets and get the sentiment
class TweetCleaner: 
    def __init__(self,stopwords_fname):
        self.stopwords_fname = stopwords_fname
        self.negation_cues = self.get_negation_cues('negation_cues.txt')
        self.sentiment_fnames = ["EffectWordNet.tff", "subjclueslen1-HLTEMNLP05.tff"]
        self.emoji_pattern = re.compile(r'[\U00010000-\U0010ffff]', flags=re.UNICODE)
        self.url_pattern = re.compile(r'https?:\/\/.*\b')
        self.handle_pattern = re.compile(r'@\w+')
        self.hashtag_pattern = re.compile(r'#\w+')
        
        #Populate the Stop Words
        self.stopwords = set()
        self.populateStopWords(self.stopwords_fname)
        
        #populate the wordtoEffectMap
        self.wordToEffectMap = {}
        self.buildWordToEffectMap(self.sentiment_fnames)
        
        #Initialize nltk classes
        self.stemmer = PorterStemmer()
        self.tokenizer = TweetTokenizer()
    
    #---------Create the Stop words Set---------#
    def populateStopWords(self,fname):
        stop_file = open(fname)
        
        for line in stop_file:
            self.stopwords.add(line.strip())
    
    def get_negation_cues(self, fname):
        cues_fname = open(fname, 'r')
        neg = []
        for line in cues_fname:
            if not line:
                continue
            line = line.strip()
            neg.append(line)
        return neg
    
    #---------Build the word to +/- Effect Map----------#
    def buildWordToEffectMap(self, sentiment_files):
        #read sentiment files into dictionary of words and positive or negative sentiment
        for file in sentiment_files:
            with open(file, "r", encoding='utf8', errors='replace') as f:
                if(file == "EffectWordNet.tff"):
                    for line in f:
                        #02279615	+Effect	profiteer	 make an unreasonable profit, as on the sale of difficult to obtain goods 
                        words = line.split('\t')
                        effect = words[1]
                        effect_val = 0
                        if '+' in effect:
                            effect_val = 1
                        elif '-' in effect:
                            effect_val = -1
                        else:
                            effect_val = 0
                        list_of_words = []
                        if (',' in words[2]):
                            list_of_words = words[2].split(',')
                        else:
                            list_of_words.append(words[2])
                        for word in list_of_words:
                            self.wordToEffectMap[word] = effect_val
                elif(file == "subjclueslen1-HLTEMNLP05.tff"):
                    for line in f:
                        #type=weaksubj len=1 word1=abandoned pos1=adj stemmed1=n priorpolarity=negative
                        line = line.strip()
                        if not line:
                            continue
                        words = line.split(' ')
                        effect_val = words[5]
                        #print(effect_val)
                        word_val = words[2]
                        word = word_val.split("=")[1]
                        effect = effect_val.split("=")[1]
                        eff = 0
                        if (effect == "positive"):
                            eff = 1
                        elif (effect == "negative"):
                            eff = -1
                        elif (effect == "neutral"):
                            eff = 0
                        self.wordToEffectMap[word] = eff
        
    #---------Clean the tweets---------#
    def cleanTweet(self,tweet):
        tweet = tweet.lower()
        
        #1. remove emojis
        tweet = re.sub(self.emoji_pattern, r'', tweet)
        
        #remove URLS
        tweet = re.sub(self.url_pattern, r'', tweet)

        #convert @mentions to AT_USER
        tweet = re.sub(self.handle_pattern, r'AT_USER', tweet)
        
        #convert #tags to HASH_TAG
        tweet = re.sub(self.hashtag_pattern, r'HASH_TAG', tweet)
        
        tweet_list = self.tokenizer.tokenize(tweet)
        
        #2. remove the stop words
        words_filtered = []
        for word in tweet_list:
            if (word not in self.stopwords ):
                words_filtered.append( word )
              
        #3. Stem the words
        #words_stemmed = [self.stemmer.stem(word) for word in words_filtered]
        
        _tweet = ""
        for word in words_filtered:
            _tweet += " " + word
        
        #4. return the fitered tweet
        return _tweet
             
    #--------- negative: 0 , neutral : 1 , positive : 2 ---------#
    def getSentiment(self,tweet):
        score = 0
        tweet = tweet.lower()
        tweet_list = self.tokenizer.tokenize(tweet)
        for word in tweet_list:
            if word in self.wordToEffectMap:
                score += self.wordToEffectMap[word]
        if (score < 0):
            return 0
        elif (score > 0):
            return 2
        else:
            return 1
        
        

In [ ]:
#Cleans all the tweets , builds the tweet set and adds the sentiment to the tweets
fw = open('flyfrontier_tweets_with_sentiment.txt', 'w', encoding='utf8', errors='replace')
class BuildFeatureSet: 
    
    def __init__(self,n,stopwords_fname,max_features_count=5000): # n : ngram for the tweets
        self.ngrams = n

        #Lexicon related variables
        self.lexicon = {}
        self.inverse_lexicon = { }
        self.ngram_count = 0
        self.max_features_count = max_features_count
        self.tweetCleaner = TweetCleaner(stopwords_fname)
        self.tweet_count = 0
        self.tweet_map = {}
        
        #Training Dataset
        self.data = {}

    def addToLexicon(self,words):
        for word in words:
            if ( word not in self.lexicon and ( self.ngram_count <  self.max_features_count ) ):
                self.lexicon[word] = self.ngram_count #Assign a unique number for the word seen
                self.inverse_lexicon[self.ngram_count] = word
                self.ngram_count = self.ngram_count + 1
                #print( 'Lexicon: ',self.ngram_count,word)
    
    def isTweetReply(self,tweet):
        if ( tweet[0].lower() == 'r' and  tweet[1].lower() == 't' ):
            return True
        else:
            return False
    
    def get_tweet_map(self):
        return self.tweet_map
        
    def isTweetFromAirline(self, tweet, airline_handle):
        if( tweet['user']['screen_name'] == airline_handle):
            return True
        else:
            return False
        
    #Add the tweet to the lexicon set
    def addTweet(self,tweet):
        
        self.tweet_count += 1
        self.tweet_map[self.tweet_count] = tweet
        
        #1. get the sentiment for the tweet
        sentiment = self.tweetCleaner.getSentiment(tweet)
        
        #2. Clean the Tweet
        tweet = self.tweetCleaner.cleanTweet(tweet)
        
        #3. get the ngrams for the tweet
        _ngrams = nltk.ngrams(tweet.split(), self.ngrams)
            
        #4. Add the ngrams to the lexicon dictionary
        words = list(_ngrams)
        self.addToLexicon( words )
        
        #5. Add this tweet row to the training set
        self.addToTrainingData(words,sentiment)
    
    #Build the Feature set for all the tweets
    def addToTrainingData(self,ngrams,sentiment):
        row = np.zeros(self.max_features_count + 1 ) # last feature is the label 
        
        for word in ngrams:
            if ( word in self.lexicon ):
                row[ self.lexicon[word] ] = row[ self.lexicon[word] ] + 1 #Increase the count of the word 

        row [ self.max_features_count ] = sentiment
        
        self.data[ len(self.data)  ]  = row 
    
    
    def getFeatures(self):
        features = np.zeros(( len(self.data), self.max_features_count ))
        
        for index in self.data:
            features[index][:] =  self.data[index][:-1]
            
        return features
    
    def getLabels(self):
        labels = np.zeros( len(self.data) )
        
        for index in self.data:
            labels[index] = ( int(self.data[index][self.max_features_count]) )
            
        return labels
    
    def getHeaders(self):
        headers = []
        for i in range(self.max_features_count):
            headers.append( self.inverse_lexicon[i] )
        return headers
            
    

In [ ]:
#Parse the Tweets
jsonParser = JsonParser()
features = BuildFeatureSet(2,'stopwords.txt') 

files = ['tweets_Allegiant.txt']#,'tweets_AlaskaAir.txt','tweets_Delta.txt','tweets_AmericanAir.txt','tweets_FlyFrontier.txt','tweets_HawaiianAir.txt'] 


for file in files:
    print ('Building Feature set for the file:',file)
    tweets =  jsonParser.loadData(file)
    airline_handle = file.split("_")[1]
    for tweet in tweets:
        #1. Skip if the tweet the is a reply to an existing tweet
        if ( features.isTweetReply(tweet['text']) == False and features.isTweetFromAirline(tweet, airline_handle) == False ):
            features.addTweet ( tweet['text']  ) 

# print ( features.getFeatures().shape )

# print ( features.getLabels().shape )

# print ( features.getHeaders() )


In [ ]:
#Logistic regression classifier

X = features.getFeatures()
y = features.getLabels()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

param_grid = {'C':[0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10, 30, 100, 300]}
logreg = LogisticRegression(C=3.0)

#clf = GridSearchCV(logreg, param_grid)
#clf.fit(X_train, y_train)

#print(clf.best_params_)
#print(clf.best_score_)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

m,n = X_train.shape
i,j = X_test.shape

twtmap = features.get_tweet_map()

#for val in range(i):
#    print ( twtmap[m+val], y_pred[val], y_test[val] )

In [ ]:

class Visualize:
    def __init__(self,debug,removeWordsList=[]):
        self.debug = debug
        
        #map[word]->count for the word cloud
        self.poscount = {} 
        self.negcount = {}  
        self.neutralcount = {} 
        self.jsonParser = JsonParser()
        self.tweetCleaner = TweetCleaner('stopwords.txt')
        self.removeWordsList = removeWordsList #List of keywords to remove for the wordcount. ex: AT_USER
    
    def log(self,msg):
        if ( self.debug ):
            print (msg)
    
    #Function to return the top n words 
    
    def getTopN(self,n,mode):# mode: 0 | 1 | 2 
        if( mode == 0):
            items = sorted(self.negcount.items(), key=lambda x: x[1],reverse = True)
        elif( mode == 1 ):
            items = sorted(self.neutralcount.items(), key=lambda x: x[1],reverse = True)
        elif( mode == 2 ):
            items = sorted(self.poscount.items(), key=lambda x: x[1],reverse = True)
        items = items[0:n]
        result = [i for i in items if i[0] not in self.removeWordsList]
        return result
    
    def writeToCSV(self,fname,n,mode):
        if(mode == 0):
            fname=fname+'_neg' +'.csv'
        elif ( mode == 1):
            fname=fname+'_neut'+'.csv'
        else:
            fname=fname+'_pos'+'.csv'
            
        with open(fname,'w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow(['word','count'])
            rows = self.getTopN(n,mode)
            for row in rows:
                csv_out.writerow(row)
        
    def setWordCount(self,fname):
        self.log('File name is,'+fname)
        tweets =  jsonParser.loadData(fname)
        
        self.log('Reading all the tweets')
        
        for tweet in tweets:
            #1.Skip for retweets
            if ( tweet['text'][0].lower() == 'r' and  tweet['text'][1].lower() == 't' ):
                continue
                
            text = self.tweetCleaner.cleanTweet( tweet['text'] )
            sentiment = self.tweetCleaner.getSentiment( tweet['text'] )
            if ( sentiment == 2):
                for  word in text.split(' '):
                    if ( word not in self.poscount ):
                        self.poscount[word] = 1
                    else:
                        self.poscount[word] = self.poscount[word] + 1
            elif ( sentiment == 1 ):
                for  word in text.split(' '):
                    if ( word not in self.neutralcount ):
                        self.neutralcount[word] = 1
                    else:
                        self.neutralcount[word] = self.neutralcount[word] + 1
            else:
                for  word in text.split(' '):
                    if ( word not in self.negcount ):
                        self.negcount[word] = 1
                    else:
                        self.negcount[word] = self.negcount[word] + 1
                    
        self.log('Word count is set')

In [ ]:
removeWordsList = ['AT_USER','HASH_TAG','...',',','’','rt','','…']
visualize = Visualize(False,removeWordsList)
visualize.setWordCount('tweets_AmericanAir.txt')

print ( 'negative', visualize.getTopN(30,0) )
print ( 'neutral',visualize.getTopN(30,1) )
print ( 'positive',visualize.getTopN(30,2) )


#Write Data to a sav file for visualisation in tableau

#visualize.writeToCSV('wc_AmericanAir',200,0)
#visualize.writeToCSV('wc_AmericanAir',200,1)
#visualize.writeToCSV('wc_AmericanAir',200,2)

